In [2]:
import pandas as pd
import numpy as np

data= pd.read_csv('dataset/kc_house_data.csv')

# QUAIS IMÓVEIS DEVE-SE COMPRAR?̣ 

In [151]:
RegionGroup= data[['price', 'zipcode']].groupby(['zipcode']).mean()
RegionGroup

,price
zipcode,
98001,2.808047e+05
98002,2.342840e+05
98003,2.941113e+05
98004,1.355927e+06
98005,8.101649e+05
...,...
98177,6.761854e+05
98178,3.106128e+05
98188,2.890783e+05


In [153]:

data['Este valor está: '] = 'NA'
for i in range(len(RegionGroup)):
    prices = data['price'].loc[data['zipcode'] == RegionGroup.index[i]]
    for row in range(len(prices)):
        idx = prices.index[row]
        
        if prices.iloc[row] >= RegionGroup['price'].iloc[i]:
            data['Este valor está: '].iloc[idx] = 'acima da média'
        else:
            data['Este valor está: '].iloc[idx] = 'abaixo da média'

In [154]:

data['Se Deve Comprar?'] = 'NÃO'

data.loc[data['Este valor está: '] == 'abaixo da média', 'Se Deve Comprar?']= 'SIM'
TabelaDeRecomendação= data[['zipcode', 'id', 'price', 'Este valor está: ', 'Se Deve Comprar?']]
TabelaDeRecomendação

,zipcode,id,price,Este valor está:,Se Deve Comprar?
0,98178,7129300520,221900.0,abaixo da média,SIM
1,98125,6414100192,538000.0,acima da média,NÃO
2,98028,5631500400,180000.0,abaixo da média,SIM
3,98136,2487200875,604000.0,acima da média,NÃO
4,98074,1954400510,510000.0,abaixo da média,SIM
...,...,...,...,...,...
21608,98103,263000018,360000.0,abaixo da média,SIM
21609,98146,6600060120,400000.0,acima da média,NÃO
21610,98144,1523300141,402101.0,abaixo da média,SIM
21611,98027,291310100,400000.0,abaixo da média,SIM


# MÉDIAS POR ESTAÇÃO DO ANO

In [ ]:
#Summer: June(6) 21st to September(9) 23rd;
#Fall: September(9) 22 or 23 to December(12) 22;
#Winter: December(12) 22nd to March(3) 20th;
#Spring: March(3) 20th to June(6) 21st.

In [134]:
data['temporadas'] = 'NA'
data['date']= pd.to_datetime(data['date'])

data.loc[(data['date'] >= '2014-06-21') & (data['date'] < '2014-09-23'), 'Temporadas'] =   'Verão'

data.loc[((data['date'] >= '2014-09-23') & (data['date'] < '2014-12-22')), 'Temporadas'] = 'Outono'

data.loc[(data['date'] >= '2014-12-22') & (data['date'] < '2015-03-20'), 'Temporadas'] =   'Inverno'

data.loc[(data['date'] < '2014-06-21') | (data['date'] >= '2015-03-20'), 'Temporadas'] =   'Primavera'

In [339]:
#The best time to sell real estate is in the spring, when properties prices are at their highest.


In [135]:

dfMedianZipcodeSeason = data[['price', 'zipcode', 'Temporadas']].groupby(['zipcode', 'Temporadas']).median().reset_index()
dfMedianZipcodeSeason.set_index('zipcode', inplace= True)
dfMedianZipcodeSeason

,Temporadas,price
zipcode,,
98001,Inverno,260000.0
98001,Outono,250000.0
98001,Primavera,265000.0
98001,Verão,262500.0
98002,Inverno,236475.0
...,...,...
98198,Verão,269800.0
98199,Inverno,604000.0
98199,Outono,650000.0


In [139]:
data['sale_price'] = 'NaN'
data['median_price'] = 'NaN'

zipcodes = list(data['zipcode'].unique())

for i in range(len(zipcodes)):
    priceAndSeason = data[['zipcode', 'price', 'Temporadas']].loc[data['zipcode'] == zipcodes[i]]
                                                       
    for row in range(len(priceAndSeason)):
        idx = priceAndSeason.index[row]
        Temporadas = priceAndSeason['Temporadas'].iloc[row]
                                                        
        dfLoc = dfMedianZipcodeSeason.loc[(dfMedianZipcodeSeason.index == zipcodes[i]) &
                        (dfMedianZipcodeSeason['Temporadas'] == Temporadas)]
                                                        
        if priceAndSeason['price'].iloc[row] > dfLoc['price'].iloc[0]:
            data['sale_price'].iloc[idx] = data['price'].iloc[idx] * 1.1
            data['median_price'].iloc[idx] = dfLoc['price'].iloc[0]
        else:
            data['sale_price'].iloc[idx] = data['price'].iloc[idx] * 1.3
            data['median_price'].iloc[idx] = dfLoc['price'].iloc[0]

In [140]:
data_sale = data[['id', 'date', 'zipcode', 'Temporadas', 'price', 'median_price', 'sale_price']]
data_sale

,id,date,zipcode,Temporadas,price,median_price,sale_price
0,7129300520,2014-10-13,98178,Outono,221900.0,284000.0,288470.0
1,6414100192,2014-12-09,98125,Outono,538000.0,435500.0,591800.0
2,5631500400,2015-02-25,98028,Inverno,180000.0,443250.0,234000.0
3,2487200875,2014-12-09,98136,Outono,604000.0,469300.0,664400.0
4,1954400510,2015-02-18,98074,Inverno,510000.0,627800.0,663000.0
...,...,...,...,...,...,...,...
21608,263000018,2014-05-21,98103,Primavera,360000.0,550000.0,468000.0
21609,6600060120,2015-02-23,98146,Inverno,400000.0,265000.0,440000.0
21610,1523300141,2014-06-23,98144,Verão,402101.0,477500.0,522731.3
21611,291310100,2015-01-16,98027,Inverno,400000.0,548500.0,520000.0


In [142]:
data_sale.rename(columns= {'price': 'buy_price'}, inplace= True)

/home/arquinho/.local/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [144]:
data_sale['gain'] = 'Null'
for i in range(len(data_sale)):
    data_sale.loc[i, 'gain'] = data_sale.loc[i, 'sale_price'] - data_sale.loc[i, 'buy_price']

/tmp/ipykernel_9887/1450526250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sale['gain'] = 'Null'


In [145]:
data_sale.head()

,id,date,zipcode,Temporadas,buy_price,median_price,sale_price,gain
0,7129300520,2014-10-13,98178,Outono,221900.0,284000.0,288470.0,66570.0
1,6414100192,2014-12-09,98125,Outono,538000.0,435500.0,591800.0,53800.0
2,5631500400,2015-02-25,98028,Inverno,180000.0,443250.0,234000.0,54000.0
3,2487200875,2014-12-09,98136,Outono,604000.0,469300.0,664400.0,60400.0
4,1954400510,2015-02-18,98074,Inverno,510000.0,627800.0,663000.0,153000.0


In [ ]:
# VALIDANDO HIPOTES

In [3]:
# H1: Imóveis que possuem vista para água, são 30% mais caros, na média.
    # Falso pois casas com vista para agua são em média 212.64% mais caras do que as sem vista para água
v1= (data.loc[data['waterfront']==1, 'price'].mean()) # = 1,661,876.0245
v2= (data.loc[data['waterfront']==0, 'price'].mean()) # 531563.5998

# Regra de 3
print(v1*100/v2-100) # 212.64% mais caro

212.6391696351833


In [4]:
# H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.
    # Falso pois as casas feitas antes de 1955 são 0.79% mais baratas
v1= (data.loc[data['yr_built']<1955, 'price'].mean()) # 537050.9076
v2=(data.loc[data['yr_built']>=1955, 'price'].mean()) #541299.9684
v1*100/v2-100 # 0.78% mais baratas

-0.7849734037613274

In [5]:
# H3: Imóveis sem porão possuem sqrt_lot, são 50% maiores do que com porão.
    # Falso pois as casas com porão sao em media 18.4% maiores do que as sem
v1= data.loc[data['sqft_basement']>0, 'sqft_lot'].mean() # 13286.29 sqft
v2= data.loc[data['sqft_basement']==0, 'sqft_lot'].mean() # 16284.17
v1*100/v2-100 # imoveis sem porao sao 18.4% maiores

-18.409784757710298

In [6]:
# H4: O crescimento do preço dos imóveis YoY ( Year OverYear ) é de 10%
    # Falso, historicamente há um decressimo de 0.59% no valor dos imóveis
df1= data[['price', 'yr_built']]

In [7]:
df1['difference']= df1['price'].diff(1)
df1.head()

/tmp/ipykernel_31436/2957546409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['difference']= df1['price'].diff(1)


,price,yr_built,difference
0,221900.0,1955,NaN
1,538000.0,1951,316100.0
2,180000.0,1933,-358000.0
3,604000.0,1965,424000.0
4,510000.0,1987,-94000.0


In [275]:
df1= df1.groupby('yr_built')['price'].mean().reset_index()

In [276]:
df1.head()

,yr_built,price
0,1900,581387.206897
1,1901,556935.931034
2,1902,673007.407407
3,1903,480958.195652
4,1904,583756.644444


In [277]:
df1['difference']= df1['price'].diff(1)
df1.head()

,yr_built,price,difference
0,1900,581387.206897,NaN
1,1901,556935.931034,-24451.275862
2,1902,673007.407407,116071.476373
3,1903,480958.195652,-192049.211755
4,1904,583756.644444,102798.448792


In [278]:
df1['YoY Percentage Diff']= round(df1['difference']/df1['price']*100,2)
df1

,yr_built,price,difference,YoY Percentage Diff
0,1900,581387.206897,NaN,NaN
1,1901,556935.931034,-24451.275862,-4.39
2,1902,673007.407407,116071.476373,17.25
3,1903,480958.195652,-192049.211755,-39.93
4,1904,583756.644444,102798.448792,17.61
...,...,...,...,...
111,2011,544522.000000,-7106.664336,-1.31
112,2012,527447.335294,-17074.664706,-3.24
113,2013,678545.452736,151098.117442,22.27
114,2014,683681.754919,5136.302183,0.75


In [279]:
df2= df1['YoY Percentage Diff'].mean()
df2

-0.5973043478260871

In [8]:
# H5: Imóveis com 3 banheiros tem um crescimento MoM ( Month over Month ) de 15%
    # O crescimento histórico entre os meses é de 0.45%
bath= data.loc[data['bathrooms']==3, ['price', 'date', 'bathrooms']]

In [9]:
bath['difference']= bath['price'].diff(1)

In [10]:
bath['date']= pd.to_datetime(bath['date'])

In [11]:
bath['Month']= bath['date'].dt.month
# bath['Month']= bath['date']

In [12]:
bath= bath.groupby('Month')['price'].mean().reset_index()

In [13]:
bath['difference']= bath['price'].diff(1)

In [14]:
bath['MoM Percentage Diff']= round(bath['difference']/bath['price']*100, 2)

In [15]:
end= bath['MoM Percentage Diff'].mean()
end

0.44999999999999996

In [288]:
data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [289]:
# H6: Há mais imóveis com vista para agua em condi'ões superiores a 3
    # Falso, ha mais casas com condi'cao 3

In [290]:
data.loc[data['waterfront']==1, 'condition'].shape #163
data.loc[(data['waterfront']==1) & (data['condition']==1)].shape #1
data.loc[(data['waterfront']==1) & (data['condition']==2)].shape #1
data.loc[(data['waterfront']==1) & (data['condition']==3)].shape #91
data.loc[(data['waterfront']==1) & (data['condition']==4)].shape #50
data.loc[(data['waterfront']==1) & (data['condition']==5)].shape #20

(20, 21)

In [291]:
# H7 Casas construidas apos os anos 2000 sao mais de 15 porcento mais caras
    # Verdadeiro, casas construidas apos os anos 2000 sao 18.78% mais caras

In [17]:
v1= data.loc[data['yr_built']>2000, 'price'].mean() #615314
v2= data.loc[data['yr_built']<2000, 'price'].mean() #517986
v1*100/v2-100

18.78968525650886

In [293]:
# H8 Casas renovadas antes de 2010 sao mais caras do que as renovadas apos
    # Falso, casas renovadas antes de 2010 sao 20.95% mais baratas

In [16]:
v1= data.loc[data['yr_renovated']<2010, 'price'].mean() #528821.95
v2= data.loc[data['yr_renovated']>2010, 'price'].mean() #684674.77
v2*100/v1-100

20.952892835225086